In [10]:
import streamlit as st
import io
from pathlib import Path
import pandas as pd
import numpy as np
import requests
import toml
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import plotly.express as px
import plotly.graph_objects as go
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

In [2]:
def fetch_and_clean_data():

    ########################  ECO2MIX  ########################
    df_eco2mix=pd.read_csv("https://drive.google.com/uc?export=download&id=1ETwJ7TnCFs4pd2BIAgI5TUQqTkLChjJ8&confirm=uuid=5e31a1d5-ac6f-46cd-9906-69170e80cbb8", sep =';', error_bad_lines=False)    
    df_eco2mix_init=df_eco2mix.head(10)
    
    #Nettoyage des données suite au résultat de la corrélation.
    df_eco2mix['Date']=pd.to_datetime(df_eco2mix['Date'])
    df_eco2mix=df_eco2mix.loc[df_eco2mix['Date'].dt.year>=2016]
    df_eco2mix=df_eco2mix.drop(['Code INSEE région','Date - Heure','Nature', 'Stockage batterie','Déstockage batterie', 'Eolien terrestre', 'Eolien offshore','TCO Thermique (%)', 'TCO Nucléaire (%)', 'TCO Eolien (%)','TCO Solaire (%)',  'TCO Hydraulique (%)', 'TCO Bioénergies (%)','Column 30'], axis=1)
    df_eco2mix=df_eco2mix.drop(['TCH Thermique (%)','TCH Nucléaire (%)','TCH Eolien (%)','TCH Solaire (%)','TCH Hydraulique (%)','TCH Bioénergies (%)'],axis=1)
    df_eco2mix=df_eco2mix.fillna(0)
    df_eco2mix['Année'] = df_eco2mix['Date'].dt.year    
    df_eco2mix['an']=df_eco2mix['Date'].dt.year
    df_eco2mix['mois']=df_eco2mix['Date'].dt.month
    df_eco2mix['jour']=df_eco2mix['Date'].dt.weekday
    
    ########################  JOURS FERIES  ########################
    ##Téléchargement et affichage du dataset des jours feriés.
    df_jour_f = pd.read_csv("D:/éCO2mix/FATOU1234/jours_feries_metropole.csv", sep=',')    
    #Nettoyage des données.
    df_jour_f.drop(['annee','zone'],axis=1,inplace=True)

    #Remplacement des jours feriés par 1.
    df_jour_f=df_jour_f.replace(to_replace=['1er janvier' ,'Lundi de Pâques' ,'1er mai' ,'8 mai', 'Ascension','Lundi de Pentecôte', '14 juillet', 'Assomption', 'Toussaint' ,'11 novembre','Jour de Noël'], value=[1,2,3,4,5,6,7,8,9,10,11])
    df_jour_f=df_jour_f.rename({'nom_jour_ferie':'jour_ferie'},axis=1)
    df_jour_f=df_jour_f.rename({'date':'Date'},axis=1)
    df_jour_f['Date']=pd.to_datetime(df_jour_f['Date'], dayfirst=True)
    df_jour_f=df_jour_f.loc[df_jour_f['Date'].dt.year>=2016]
  
    ########################  TEMPÉRATURES  ########################
    #Chargement des données du data set températures.
    temp=pd.read_csv("D:/éCO2mix/FATOU1234/temperature-quotidienne-regionale.csv", sep=';')

    #Suppression de la variable 'Code INSEE région'.
    temp=temp.drop('Code INSEE région',axis=1)

    #Conversion des dates en format datetime.
    temp['Date']=pd.to_datetime(temp['Date'], dayfirst=True)
    temp=temp.loc[temp['Date'].dt.year>=2016]
    temp=temp.sort_values(by='Date')
  
    return df_eco2mix_init,df_eco2mix, df_jour_f,temp

In [3]:
df_eco2mix_init,df,jour_f,temp=fetch_and_clean_data()

C:\Users\mousb\AppData\Local\Temp\ipykernel_33324\761062342.py:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df_eco2mix=pd.read_csv("https://drive.google.com/uc?export=download&id=1ETwJ7TnCFs4pd2BIAgI5TUQqTkLChjJ8&confirm=uuid=5e31a1d5-ac6f-46cd-9906-69170e80cbb8", sep =';', error_bad_lines=False)


In [8]:
display(df_eco2mix_init.head(10))

display(df.head(10))

display(jour_f.head(10))

display(temp.head(10))

,Code INSEE région,Région,Nature,Date,Heure,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),...,TCH Nucléaire (%),TCO Eolien (%),TCH Eolien (%),TCO Solaire (%),TCH Solaire (%),TCO Hydraulique (%),TCH Hydraulique (%),TCO Bioénergies (%),TCH Bioénergies (%),Column 30
0,27,Bourgogne-Franche-Comté,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28,Normandie,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,Hauts-de-France,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,53,Bretagne,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,75,Nouvelle-Aquitaine,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,11,Île-de-France,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,84,Auvergne-Rhône-Alpes,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,76,Occitanie,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,24,Centre-Val de Loire,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,44,Grand Est,Données définitives,2013-01-01,00:00,2013-01-01T00:00:00+01:00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Région,Date,Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Pompage (MW),Bioénergies (MW),Ech. physiques (MW),Année,an,mois,jour
630720,Bourgogne-Franche-Comté,2016-01-01,00:00,2456.0,117.0,0.0,85.0,0.0,23.0,0.0,19.0,2213.0,2016,2016,1,4
630721,Nouvelle-Aquitaine,2016-01-01,00:00,5570.0,118.0,6532.0,143.0,-1.0,82.0,0.0,184.0,-1488.0,2016,2016,1,4
630722,Île-de-France,2016-01-01,00:00,8885.0,293.0,0.0,18.0,0.0,10.0,0.0,137.0,8428.0,2016,2016,1,4
630723,Provence-Alpes-Côte d'Azur,2016-01-01,00:00,5858.0,412.0,0.0,1.0,0.0,223.0,0.0,83.0,5139.0,2016,2016,1,4
630724,Hauts-de-France,2016-01-01,00:00,5969.0,510.0,5223.0,882.0,0.0,2.0,0.0,136.0,-784.0,2016,2016,1,4
630725,Grand Est,2016-01-01,00:00,5130.0,169.0,10941.0,866.0,0.0,449.0,-62.0,73.0,-7307.0,2016,2016,1,4
630726,Centre-Val de Loire,2016-01-01,00:00,2446.0,75.0,11237.0,347.0,0.0,1.0,0.0,54.0,-9268.0,2016,2016,1,4
630727,Normandie,2016-01-01,00:00,3582.0,325.0,6452.0,143.0,0.0,14.0,0.0,55.0,-3405.0,2016,2016,1,4
630728,Occitanie,2016-01-01,00:00,5228.0,60.0,2476.0,47.0,0.0,840.0,0.0,101.0,1704.0,2016,2016,1,4
630729,Bretagne,2016-01-01,00:00,3344.0,140.0,0.0,5.0,0.0,3.0,-50.0,38.0,3208.0,2016,2016,1,4


,Date,jour_ferie
154,2016-01-01,1
155,2016-03-28,2
156,2016-05-01,3
157,2016-05-05,5
158,2016-05-08,4
159,2016-05-16,6
160,2016-07-14,7
161,2016-08-15,8
162,2016-11-01,9
163,2016-11-11,10


,ID,Date,Région,TMin (°C),TMax (°C),TMoy (°C)
22351,2016-01-01T00:00:00-75,2016-01-01,Nouvelle-Aquitaine,3.32,13.58,8.45
22350,2016-01-01T00:00:00-11,2016-01-01,Île-de-France,3.30,8.76,6.03
22352,2016-01-01T00:00:00-93,2016-01-01,Provence-Alpes-Côte d'Azur,5.30,12.77,9.03
25245,2016-01-01T00:00:00-94,2016-01-01,Corse,9.95,15.10,12.52
5799,2016-01-01T00:00:00-53,2016-01-01,Bretagne,3.00,12.27,7.64
22349,2016-01-01T00:00:00-24,2016-01-01,Centre-Val de Loire,1.52,10.73,6.12
5798,2016-01-01T00:00:00-28,2016-01-01,Normandie,2.78,9.40,6.09
5797,2016-01-01T00:00:00-76,2016-01-01,Occitanie,4.54,13.34,8.94
5796,2016-01-01T00:00:00-84,2016-01-01,Auvergne-Rhône-Alpes,2.96,10.56,6.76
25244,2016-01-01T00:00:00-44,2016-01-01,Grand Est,1.13,6.99,4.06


In [9]:
    display("Modélisation")
    display("Afin d'éviter le temps d'entraînement des modèles, nous les avons entraîné et enregistré en amont à l'aide du module **pickle**")
    
    temp_IDF=temp[temp["Région"]=='Île-de-France']
    data_IDF=df[df['Région']=='Île-de-France']

    #Merge des deux data set.
    data_IDF=pd.merge(data_IDF,jour_f,on='Date',how='left')

    #Rempacement des nans par des zéros.
    data_IDF=data_IDF.fillna(0)
    data_IDF=data_IDF.groupby(['Date','Région','jour','mois','jour_ferie'],as_index=False).agg({'Consommation (MW)':'sum'})

    #Création d'une nouvelle variable 'Consommation J-7 (MW)'.
    data_IDF['Consommation J-7 (MW)']=data_IDF['Consommation (MW)'].shift(7,axis=0)
    #Création d'une nouvelle variable 'Consommation J-30 (MW)'.
    data_IDF['Consommation J-30 (MW)']=data_IDF['Consommation (MW)'].shift(30,axis=0)

    #Fusion des dataframe consommation et température.
    data_IDF=temp_IDF.merge(right=data_IDF, on=["Date","Région"], how='inner')
   
    #Supression des variables catégorielles.
    data_IDF.drop(["ID","Région",'Date'], axis=1, inplace=True)
    
    #Supression de toutes les lignes contenant des valeurs manquantes :
    data_IDF = data_IDF[(data_IDF.isna().sum(axis=1)) < 1]
    
    #Instanciation de la variable target et les variables explicatives.

    X=data_IDF.drop('Consommation (MW)',axis=1)
    Y=data_IDF['Consommation (MW)']
    
    #Construction des jeux de données d'entraînement et de test

    X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=0.2,shuffle=False,random_state=66)
    
   # display(data_IDF.head(10))
   # 
   # plt.figure(figsize=(10,10))
   # sns.pairplot(data_IDF, diag_kind='kde');
   # plt.title('Distibution et relations entre les variables');

'Modélisation'

"Afin d'éviter le temps d'entraînement des modèles, nous les avons entraîné et enregistré en amont à l'aide du module **pickle**"

In [13]:
#On va tester les différents modèles de regression et nous comparons ensuite les performances.

#Test des différents modèles de régression.
#Modèle 1: LinearRegression.
########################################################################################################
#########################                 LinearRegression                     #########################
########################################################################################################
lr = LinearRegression()

#Ajustement du modèle à l'ensemble d'entraînement.

lr.fit(X_train,Y_train)

filename = 'LinearRegression.joblib'
pickle.dump(lr, open(filename, 'wb'))

#Modèle 2: RandomForestRegressor.
########################################################################################################
#########################                 RandomForestRegressor                #########################
########################################################################################################
rf = RandomForestRegressor(random_state=42)

#Ajustement du modèle à l'ensemble d'entraînement.

rf.fit(X_train,Y_train)

filename = 'RandomForestRegressor.joblib'
pickle.dump(rf, open(filename, 'wb'))

#Modèle 3: KNeighborsRegressor.
########################################################################################################
#########################                 KNeighborsRegressor                  #########################
########################################################################################################
knn = KNeighborsRegressor(n_neighbors=6)

#Ajustement du modèle à l'ensemble d'entraînement.

knn.fit(X_train,Y_train)

filename = 'KNeighborsRegressor.joblib'
pickle.dump(knn, open(filename, 'wb'))

#Modèle 4: GradientBoostingClassifier.
########################################################################################################
#########################                 GradientBoostingRegressor            #########################
########################################################################################################
GBR = GradientBoostingRegressor(random_state=0)
GBR.fit(X_train, Y_train)

filename = 'GradientBoostingRegressor.joblib'
pickle.dump(GBR, open(filename, 'wb'))